# Multi-Database Agent System: PostgreSQL & CosmosDB NoSQL

このノートブックでは、PostgreSQLとCosmosDB NoSQLの両方に対応する4体のエージェントシステムを実装します：

1. **PostgreSQL Query Generation Agent** - PostgreSQL用SQLクエリ生成
2. **PostgreSQL Query Execution Agent** - PostgreSQL用SQLクエリ実行
3. **CosmosDB Query Generation Agent** - CosmosDB NoSQL用SQLクエリ生成
4. **CosmosDB Query Execution Agent** - CosmosDB NoSQL用SQLクエリ実行

これらのエージェントがGroup Chatを通じて連携し、ユーザーの自然言語質問を適切なデータベースクエリに変換して実行します。

# ライブラリのインポート

In [1]:
import os
import json
import asyncio
import datetime
import psycopg2

from dotenv import load_dotenv, find_dotenv
from typing import Annotated, Any, List, Optional
from pydantic import BaseModel

from azure.identity import DefaultAzureCredential
from azure.cosmos import CosmosClient
from azure.cosmos.exceptions import CosmosHttpResponseError

from semantic_kernel.agents import (
    Agent, ChatCompletionAgent, GroupChatOrchestration, 
    GroupChatManager, BooleanResult, StringResult, MessageResult
)
from semantic_kernel.agents.runtime import InProcessRuntime
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureChatPromptExecutionSettings
from semantic_kernel.contents import (
    ChatMessageContent, FunctionCallContent, FunctionResultContent, 
    AuthorRole, TextContent, ChatHistory
)
from semantic_kernel.functions import kernel_function
from semantic_kernel.functions.kernel_arguments import KernelArguments

print("ライブラリのインポートが完了しました。")

ライブラリのインポートが完了しました。


# 環境変数の設定

In [2]:
load_dotenv(override=True)

# Azure OpenAI 設定
PROJECT_ENDPOINT = os.getenv("PROJECT_ENDPOINT")
AZURE_DEPLOYMENT_NAME = os.getenv("AZURE_DEPLOYMENT_NAME")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")

# PostgreSQL 設定
PG_HOST = os.getenv("PG_HOST")
PG_PORT = os.getenv("PG_PORT", "5432")
PG_DB = os.getenv("PG_DB")
PG_USER = os.getenv("PG_USER")
PG_PASS = os.getenv("PG_PASS")

# CosmosDB 設定
COSMOS_ENDPOINT = os.getenv("COSMOS_ENDPOINT")
COSMOS_KEY = os.getenv("COSMOS_KEY")
COSMOS_DB = os.getenv("COSMOS_DB")
COSMOS_CONTAINER = os.getenv("COSMOS_CONTAINER")

print("環境変数の読み込みが完了しました。")
print(f"PostgreSQL Host: {PG_HOST}")
print(f"CosmosDB Endpoint: {COSMOS_ENDPOINT}")
print(f"Azure OpenAI Endpoint: {AZURE_OPENAI_ENDPOINT}")

環境変数の読み込みが完了しました。
PostgreSQL Host: pgserver-3aw-8865.postgres.database.azure.com
CosmosDB Endpoint: https://cosmos-3aw-8865.documents.azure.com:443/
Azure OpenAI Endpoint: https://foundry-3aw-8865.cognitiveservices.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2025-01-01-preview


# データベース接続設定

In [3]:
# PostgreSQL 接続設定
PG_CONFIG = {
    "user": PG_USER,
    "password": PG_PASS,
    "dbname": PG_DB,
    "host": PG_HOST,
    "port": int(PG_PORT),
}

# CosmosDB 接続設定
COSMOS_CONFIG = {
    "endpoint": COSMOS_ENDPOINT,
    "key": COSMOS_KEY,
    "database": COSMOS_DB,
    "container": COSMOS_CONTAINER
}

print("データベース接続設定が完了しました。")

データベース接続設定が完了しました。


# プラグインクラスの定義

## PostgreSQL スキーマプラグイン

In [4]:
class PostgresSchemaPlugin:
    """
    PostgreSQLのテーブル一覧・スキーマ取得専用プラグイン
    """
    def __init__(self):
        self.pg_config = PG_CONFIG

    def _get_connection(self):
        return psycopg2.connect(**self.pg_config)

    @kernel_function(
        name="get_tables",
        description="PostgreSQLデータベース内のテーブル一覧をJSON文字列で取得します。"
    )
    def get_tables(
        self,
    ) -> Annotated[str, "テーブル一覧を含むJSON文字列（例: {'tables': ['table1', 'table2']}）"]:
        with self._get_connection() as conn, conn.cursor() as cur:
            cur.execute("SELECT tablename FROM pg_tables WHERE schemaname = 'public';")
            tables = [row[0] for row in cur.fetchall()]
        return json.dumps({"tables": tables})

    @kernel_function(
        name="get_table_schema",
        description="指定したテーブルのスキーマ情報（カラム名、型など）をJSON文字列で取得します。"
    )
    def get_table_schema(
        self,
        table_name: Annotated[str, "スキーマ情報を取得したいテーブル名"]
    ) -> Annotated[str, "カラム情報を含むJSON文字列（例: {'columns': [{'name': 'id', 'type': 'integer'}, ...]}）"]:
        with self._get_connection() as conn, conn.cursor() as cur:
            cur.execute("""
                SELECT column_name, data_type
                FROM information_schema.columns
                WHERE table_name = %s;
            """, (table_name,))
            columns = [{"name": row[0], "type": row[1]} for row in cur.fetchall()]
        return json.dumps({"columns": columns})

print("PostgreSQLスキーマプラグインが定義されました。")

PostgreSQLスキーマプラグインが定義されました。


## PostgreSQL クエリ実行プラグイン

In [5]:
class PostgresQueryPlugin:
    """
    PostgreSQLのSQL実行専用プラグイン
    """
    def __init__(self):
        self.pg_config = PG_CONFIG

    def _get_connection(self):
        return psycopg2.connect(**self.pg_config)

    @kernel_function(
        name="execute_sql",
        description="任意のSQL文を実行し、結果をJSON文字列で返します。（SELECTのみ対応を推奨）"
    )
    def execute_sql(
        self,
        sql: Annotated[str, "実行したいSQLクエリ（例: 'SELECT * FROM users'）"]
    ) -> Annotated[str, "クエリ結果のJSON文字列（例: {'rows': [...]}）"]:
        try:
            print(f"Executing PostgreSQL SQL: {sql}")
            with self._get_connection() as conn, conn.cursor() as cur:
                cur.execute(sql)
                columns = [desc[0] for desc in cur.description] if cur.description else []
                rows = [dict(zip(columns, row)) for row in cur.fetchall()] if columns else []
            print(f"PostgreSQL SQL executed successfully. Rows returned: {len(rows)}")
            return json.dumps({"rows": rows, "row_count": len(rows)})
        except Exception as e:
            error_msg = str(e)
            print(f"PostgreSQL SQL execution failed: {error_msg}")
            raise Exception(error_msg)

print("PostgreSQLクエリプラグインが定義されました。")

PostgreSQLクエリプラグインが定義されました。


## CosmosDB スキーマプラグイン

In [6]:
class CosmosSchemaPlugin:
    """
    CosmosDB NoSQLのコンテナー情報取得専用プラグイン
    """
    def __init__(self):
        self.endpoint = COSMOS_CONFIG["endpoint"]
        self.key = COSMOS_CONFIG["key"]
        self.database_name = COSMOS_CONFIG["database"]
        self.container_name = COSMOS_CONFIG["container"]
        self.client = CosmosClient(self.endpoint, self.key)
        self.database = self.client.get_database_client(self.database_name)
        self.container = self.database.get_container_client(self.container_name)

    @kernel_function(
        name="get_container_info",
        description="CosmosDBコンテナーの基本情報をJSON文字列で取得します。"
    )
    def get_container_info(
        self,
    ) -> Annotated[str, "コンテナー情報を含むJSON文字列（例: {'container': 'container_name', 'database': 'db_name'}）"]:
        try:
            container_properties = self.container.read()
            info = {
                "container": self.container_name,
                "database": self.database_name,
                "partition_key": container_properties.get("partitionKey", {}),
                "indexing_policy": container_properties.get("indexingPolicy", {})
            }
            return json.dumps(info, ensure_ascii=False)
        except Exception as e:
            error_msg = f"CosmosDB container info retrieval failed: {str(e)}"
            print(error_msg)
            raise Exception(error_msg)

    @kernel_function(
        name="get_sample_documents",
        description="CosmosDBコンテナーからサンプルドキュメントを取得し、スキーマ構造を把握します。"
    )
    def get_sample_documents(
        self,
        limit: Annotated[int, "取得するサンプルドキュメント数（デフォルト: 5）"] = 5
    ) -> Annotated[str, "サンプルドキュメントを含むJSON文字列"]:
        try:
            print(f"Fetching {limit} sample documents from CosmosDB")
            query = f"SELECT TOP {limit} * FROM c"
            query_iter = self.container.query_items(
                query=query,
                enable_cross_partition_query=True
            )
            samples = [dict(item) for item in query_iter]
            print(f"CosmosDB sample documents fetched successfully. Count: {len(samples)}")
            return json.dumps({"samples": samples, "count": len(samples)}, ensure_ascii=False)
        except Exception as e:
            error_msg = f"CosmosDB sample document retrieval failed: {str(e)}"
            print(error_msg)
            raise Exception(error_msg)

print("CosmosDBスキーマプラグインが定義されました。")

CosmosDBスキーマプラグインが定義されました。


## CosmosDB クエリ実行プラグイン

In [7]:
class CosmosQueryPlugin:
    """
    CosmosDB NoSQL SQLクエリ実行専用プラグイン
    """
    def __init__(self):
        self.endpoint = COSMOS_CONFIG["endpoint"]
        self.key = COSMOS_CONFIG["key"]
        self.database_name = COSMOS_CONFIG["database"]
        self.container_name = COSMOS_CONFIG["container"]
        self.client = CosmosClient(self.endpoint, self.key)
        self.database = self.client.get_database_client(self.database_name)
        self.container = self.database.get_container_client(self.container_name)

    @kernel_function(
        name="execute_cosmos_sql",
        description="CosmosDBにSQLクエリ（SELECT系）を実行し、結果をJSON文字列で返します"
    )
    def execute_cosmos_sql(
        self,
        sql: Annotated[str, "CosmosDBのSQLクエリ（例: 'SELECT * FROM c WHERE c.status=\"active\"' ）"]
    ) -> Annotated[str, "クエリ結果のJSON文字列（例: {'rows': [...]} ）"]:
        try:
            print(f"Executing CosmosDB SQL: {sql}")
            query_iter = self.container.query_items(
                query=sql,
                enable_cross_partition_query=True
            )
            rows = [dict(item) for item in query_iter]
            print(f"CosmosDB SQL executed successfully. Rows returned: {len(rows)}")
            return json.dumps({"rows": rows, "row_count": len(rows)}, ensure_ascii=False)
        except CosmosHttpResponseError as e:
            error_msg = f"CosmosDB error: {str(e)}"
            print(error_msg)
            raise Exception(error_msg)
        except Exception as e:
            error_msg = f"CosmosDB query execution failed: {str(e)}"
            print(error_msg)
            raise Exception(error_msg)

print("CosmosDBクエリプラグインが定義されました。")

CosmosDBクエリプラグインが定義されました。


# エージェントの作成と設定

In [8]:
# Azure OpenAI サービスの初期化
azure_completion_service = AzureChatCompletion(
    service_id="azure_completion_agent",
    deployment_name=AZURE_DEPLOYMENT_NAME,
    endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY
)

# 構造化出力の設定
settings = AzureChatPromptExecutionSettings()
response_format_dict = {
    "type": "json_schema",
    "json_schema": {
        "name": "QueryExecutionResult",
        "schema": {
            "type": "object",
            "properties": {
                "status": {
                    "type": "string",
                    "enum": ["success", "error"]
                },
                "result": {
                    "type": "string",
                    "description": "JSON string containing the query execution result or error message"
                }
            },
            "required": ["status", "result"],
            "additionalProperties": False
        },
        "strict": True
    }
}
settings.response_format = response_format_dict

print("Azure OpenAIサービスと構造化出力設定が完了しました。")

Azure OpenAIサービスと構造化出力設定が完了しました。


## PostgreSQL エージェント

In [9]:
# PostgreSQL クエリ生成エージェント
postgres_query_generation_agent = ChatCompletionAgent(
    name="PostgresSQLGenerationAgent",
    description="ユーザーの自然言語質問からPostgreSQL用のSQLクエリを生成し、必要に応じて修正も行うエージェントです。",
    instructions=(
        "あなたはユーザーの質問に基づいて、PostgreSQLデータベースで実行可能なSQLクエリ（主にSELECT文）を生成する役割です。\n"
        "【必ず守ること】\n"
        "1. SQLクエリを生成する前に、まずデータベース内のテーブル一覧を取得してください。\n"
        "2. 次に、関連しそうなテーブルごとにカラム情報も取得し、スキーマ構成を十分に把握してください。\n"
        "3. 取得したテーブル・カラム情報（スキーマ情報）を必ず参照したうえで、PostgreSQL用の正しいSQLを生成してください。\n"
        "4. SQLを生成した後、PostgreSQL実行エージェントによって実行されます。\n"
        "5. もしSQL実行時にエラー（テーブルやカラムが存在しない、構文エラーなど）が発生した場合は、エラーメッセージとスキーマ情報を再確認し、原因を特定してSQLを修正してください。\n"
        "6. 不明点やスキーマに疑問がある場合は、ツールを活用して追加で情報取得を行い、十分に情報を得てから再度SQLを生成してください。\n"
        "7. 最終的に正しいSQLが完成したら、そのSQL文のみを返してください。\n"
        "【注意事項】\n"
        "・SQLインジェクションや危険なクエリ生成は厳禁です。\n"
        "・不要なコメントや説明文は出力せず、SQL文のみを返してください。\n"
        "・必ず実際のテーブル名とカラム名を確認してからSQLを生成してください。"
    ),
    service=azure_completion_service,
    plugins=[PostgresSchemaPlugin()]
)

# PostgreSQL クエリ実行エージェント
postgres_query_execution_agent = ChatCompletionAgent(
    service=AzureChatCompletion(
        service_id="azure_postgres_execution_agent",
        deployment_name=AZURE_DEPLOYMENT_NAME,
        endpoint=AZURE_OPENAI_ENDPOINT,
        api_key=AZURE_OPENAI_API_KEY
    ),
    name="PostgresSQLExecutionAgent",
    description="PostgreSQLのSQL文を実行し、構造化JSONで返すエージェント。",
    instructions=(
        "あなたは与えられたSQLクエリを、必ずそのままPostgreSQLで実行する役割です。\n"
        "【実行手順】\n"
        "1. 提供されたSQLクエリを、execute_sql関数を使用して実行してください。\n"
        "2. execute_sql関数の結果は JSON文字列形式で返されます。\n"
        "3. 実行結果に基づいて、以下の構造化JSON形式で必ず回答してください：\n\n"
        "成功時:\n"
        "{\n"
        '  "status": "success",\n'
        '  "result": "execute_sql関数から返されたJSON文字列をそのまま格納"\n'
        "}\n\n"
        "エラー時:\n"
        "{\n"
        '  "status": "error",\n'
        '  "result": "具体的なエラーメッセージ"\n'
        "}\n\n"
        "【重要】\n"
        "・必ずexecute_sql関数を呼び出してSQLを実行してください。\n"
        "・execute_sql関数の結果（JSON文字列）をresultフィールドにそのまま文字列として格納してください。\n"
        "・危険なSQL（データ破壊やセキュリティリスクを伴うもの）は実行せず、status を error にして理由をresultに記載してください。\n"
        "・構造化JSONのみを返し、余計な説明やコメントは不要です。"
    ),
    plugins=[PostgresQueryPlugin()],
    arguments=KernelArguments(settings=settings)
)

print("PostgreSQLエージェントが作成されました。")

PostgreSQLエージェントが作成されました。


## CosmosDB エージェント

In [10]:
# CosmosDB クエリ生成エージェント
cosmos_query_generation_agent = ChatCompletionAgent(
    name="CosmosDBSQLGenerationAgent",
    description="ユーザーの自然言語質問からCosmosDB NoSQL用のSQLクエリを生成し、必要に応じて修正も行うエージェントです。",
    instructions=(
        "あなたはユーザーの質問に基づいて、CosmosDB NoSQLで実行可能なSQLクエリ（主にSELECT文）を生成する役割です。\n"
        "【必ず守ること】\n"
        "1. SQLクエリを生成する前に、まずCosmosDBコンテナーの基本情報を取得してください。\n"
        "2. 次に、サンプルドキュメントを取得し、ドキュメント構造とフィールドを十分に把握してください。\n"
        "3. 取得したコンテナー情報とサンプルドキュメント構造を必ず参照したうえで、CosmosDB NoSQL用の正しいSQLを生成してください。\n"
        "4. CosmosDB NoSQLでは、'c'をコンテナーのエイリアスとして使用し、フィールドアクセスは'c.fieldname'形式で行ってください。\n"
        "5. SQLを生成した後、CosmosDB実行エージェントによって実行されます。\n"
        "6. もしSQL実行時にエラー（フィールドが存在しない、構文エラーなど）が発生した場合は、エラーメッセージとドキュメント構造を再確認し、原因を特定してSQLを修正してください。\n"
        "7. 不明点やドキュメント構造に疑問がある場合は、ツールを活用して追加で情報取得を行い、十分に情報を得てから再度SQLを生成してください。\n"
        "8. 最終的に正しいSQLが完成したら、そのSQL文のみを返してください。\n"
        "【CosmosDB NoSQL特有の注意事項】\n"
        "・文字列の比較では二重引用符を使用してください（例: c.status = \"active\"）\n"
        "・クロスパーティションクエリが必要な場合があることを理解してください\n"
        "・必ず実際のフィールド名を確認してからSQLを生成してください\n"
        "・NoSQLドキュメントの階層構造に注意してアクセスしてください（例: c.address.city）"
    ),
    service=azure_completion_service,
    plugins=[CosmosSchemaPlugin()]
)

# CosmosDB クエリ実行エージェント
cosmos_query_execution_agent = ChatCompletionAgent(
    service=AzureChatCompletion(
        service_id="azure_cosmos_execution_agent",
        deployment_name=AZURE_DEPLOYMENT_NAME,
        endpoint=AZURE_OPENAI_ENDPOINT,
        api_key=AZURE_OPENAI_API_KEY
    ),
    name="CosmosDBSQLExecutionAgent",
    description="CosmosDB NoSQLのSQL文を実行し、構造化JSONで返すエージェント。",
    instructions=(
        "あなたは与えられたSQLクエリを、必ずそのままCosmosDB NoSQLで実行する役割です。\n"
        "【実行手順】\n"
        "1. 提供されたSQLクエリを、execute_cosmos_sql関数を使用して実行してください。\n"
        "2. execute_cosmos_sql関数の結果は JSON文字列形式で返されます。\n"
        "3. 実行結果に基づいて、以下の構造化JSON形式で必ず回答してください：\n\n"
        "成功時:\n"
        "{\n"
        '  "status": "success",\n'
        '  "result": "execute_cosmos_sql関数から返されたJSON文字列をそのまま格納"\n'
        "}\n\n"
        "エラー時:\n"
        "{\n"
        '  "status": "error",\n'
        '  "result": "具体的なエラーメッセージ"\n'
        "}\n\n"
        "【重要】\n"
        "・必ずexecute_cosmos_sql関数を呼び出してSQLを実行してください。\n"
        "・execute_cosmos_sql関数の結果（JSON文字列）をresultフィールドにそのまま文字列として格納してください。\n"
        "・危険なクエリ（データ破壊やセキュリティリスクを伴うもの）は実行せず、status を error にして理由をresultに記載してください。\n"
        "・構造化JSONのみを返し、余計な説明やコメントは不要です。"
    ),
    plugins=[CosmosQueryPlugin()],
    arguments=KernelArguments(settings=settings)
)

print("CosmosDBエージェントが作成されました。")

CosmosDBエージェントが作成されました。


# マルチデータベース グループチャットマネージャー

In [11]:
# magenticオーケストレーション用のインポート
from semantic_kernel.agents import (
    MagenticOrchestration,
    StandardMagenticManager,
)

print("magenticオーケストレーション用のライブラリがインポートされました。")

magenticオーケストレーション用のライブラリがインポートされました。


In [12]:
async def create_database_agents():
    """
    PostgreSQLとCosmosDBの4体のエージェントを作成する関数
    """
    
    # PostgreSQL クエリ生成エージェント
    postgres_query_generation_agent = ChatCompletionAgent(
        name="PostgresGenerationAgent",  # 名前を短縮・明確化
        description="ユーザーの自然言語質問からPostgreSQL用のSQLクエリを生成するエージェント",
        instructions=(
            "あなたはユーザーの質問に基づいて、PostgreSQLデータベースで実行可能なSQLクエリ（主にSELECT文）を生成する役割です。\n"
            "【必ず守ること】\n"
            "1. SQLクエリを生成する前に、まずデータベース内のテーブル一覧を取得してください。\n"
            "2. 次に、関連しそうなテーブルごとにカラム情報も取得し、スキーマ構成を十分に把握してください。\n"
            "3. 取得したテーブル・カラム情報（スキーマ情報）を必ず参照したうえで、PostgreSQL用の正しいSQLを生成してください。\n"
            "4. ユーザーの質問がPostgreSQLに関連する場合のみ、SQLクエリを生成してください。\n"
            "5. CosmosDBに関する質問の場合は、他のエージェントに任せてください。\n"
            "6. 最終的に正しいSQLが完成したら、そのSQL文のみを返してください。\n"
            "【注意事項】\n"
            "・SQLインジェクションや危険なクエリ生成は厳禁です。\n"
            "・不要なコメントや説明文は出力せず、SQL文のみを返してください。\n"
            "・必ず実際のテーブル名とカラム名を確認してからSQLを生成してください。"
        ),
        service=azure_completion_service,
        plugins=[PostgresSchemaPlugin()]
    )

    # PostgreSQL クエリ実行エージェント
    postgres_query_execution_agent = ChatCompletionAgent(
        service=azure_completion_service,
        name="PostgresExecutionAgent",  # 名前を短縮・明確化
        description="PostgreSQLのSQL文を実行し、結果を返すエージェント",
        instructions=(
            "あなたは与えられたSQLクエリを、必ずそのままPostgreSQLで実行する役割です。\n"
            "【実行手順】\n"
            "1. PostgreSQL用のSQLクエリが提供された場合、execute_sql関数を使用して実行してください。\n"
            "2. execute_sql関数の結果をそのまま返してください。\n"
            "3. CosmosDBに関するクエリの場合は、他のエージェントに任せてください。\n"
            "【重要】\n"
            "・必ずexecute_sql関数を呼び出してSQLを実行してください。\n"
            "・危険なSQL（データ破壊やセキュリティリスクを伴うもの）は実行しないでください。\n"
            "・結果を分かりやすく表示してください。"
        ),
        plugins=[PostgresQueryPlugin()]
    )

    # CosmosDB クエリ生成エージェント
    cosmos_query_generation_agent = ChatCompletionAgent(
        name="CosmosGenerationAgent",  # 名前を短縮・明確化
        description="ユーザーの自然言語質問からCosmosDB NoSQL用のSQLクエリを生成するエージェント",
        instructions=(
            "あなたはユーザーの質問に基づいて、CosmosDB NoSQLで実行可能なSQLクエリ（主にSELECT文）を生成する役割です。\n"
            "【必ず守ること】\n"
            "1. SQLクエリを生成する前に、まずCosmosDBコンテナーの基本情報を取得してください。\n"
            "2. 次に、サンプルドキュメントを取得し、ドキュメント構造とフィールドを十分に把握してください。\n"
            "3. 取得したコンテナー情報とサンプルドキュメント構造を必ず参照したうえで、CosmosDB NoSQL用の正しいSQLを生成してください。\n"
            "4. CosmosDB NoSQLでは、'c'をコンテナーのエイリアスとして使用し、フィールドアクセスは'c.fieldname'形式で行ってください。\n"
            "5. ユーザーの質問がCosmosDBに関連する場合のみ、SQLクエリを生成してください。\n"
            "6. PostgreSQLに関する質問の場合は、他のエージェントに任せてください。\n"
            "7. 最終的に正しいSQLが完成したら、そのSQL文のみを返してください。\n"
            "【CosmosDB NoSQL特有の注意事項】\n"
            "・文字列の比較では二重引用符を使用してください（例: c.status = \"active\"）\n"
            "・クロスパーティションクエリが必要な場合があることを理解してください\n"
            "・必ず実際のフィールド名を確認してからSQLを生成してください\n"
            "・NoSQLドキュメントの階層構造に注意してアクセスしてください（例: c.address.city）"
        ),
        service=azure_completion_service,
        plugins=[CosmosSchemaPlugin()]
    )

    # CosmosDB クエリ実行エージェント
    cosmos_query_execution_agent = ChatCompletionAgent(
        service=azure_completion_service,
        name="CosmosExecutionAgent",  # 名前を短縮・明確化
        description="CosmosDB NoSQLのSQL文を実行し、結果を返すエージェント",
        instructions=(
            "あなたは与えられたSQLクエリを、必ずそのままCosmosDB NoSQLで実行する役割です。\n"
            "【実行手順】\n"
            "1. CosmosDB NoSQL用のSQLクエリが提供された場合、execute_cosmos_sql関数を使用して実行してください。\n"
            "2. execute_cosmos_sql関数の結果をそのまま返してください。\n"
            "3. PostgreSQLに関するクエリの場合は、他のエージェントに任せてください。\n"
            "【重要】\n"
            "・必ずexecute_cosmos_sql関数を呼び出してSQLを実行してください。\n"
            "・危険なクエリ（データ破壊やセキュリティリスクを伴うもの）は実行しないでください。\n"
            "・結果を分かりやすく表示してください。"
        ),
        plugins=[CosmosQueryPlugin()]
    )

    return [
        postgres_query_generation_agent,
        postgres_query_execution_agent,
        cosmos_query_generation_agent,
        cosmos_query_execution_agent
    ]

print("データベースエージェント作成関数が定義されました。")

データベースエージェント作成関数が定義されました。


In [ ]:
async def run_magentic_database_orchestration(user_question: str):
    """
    magenticオーケストレーションを使って4体のデータベースエージェントを協調させる関数
    """
    print(f"ユーザー質問: {user_question}")
    print("=" * 60)
    
    # エージェント作成
    agents = await create_database_agents()
    print(f"✅ {len(agents)}体のデータベースエージェントが作成されました。")
    
    # エージェント名を確認・表示
    agent_names = [agent.name for agent in agents]
    print(f"作成されたエージェント:")
    for agent in agents:
        print(f"   - {agent.name}: {agent.description}")
    
    # # 名前の重複チェック
    # if len(agent_names) != len(set(agent_names)):
    #     print("警告: エージェント名に重複があります")
    #     duplicates = [name for name in agent_names if agent_names.count(name) > 1]
    #     print(f"重複名: {set(duplicates)}")
    
    try:
        # マネージャー作成時に一意の名前を設定（エージェント名と重複しない名前）
        manager_name = "OrchestratorManager"
        while manager_name in agent_names:
            manager_name = f"Manager_{len(agent_names)}"
        
        manager = StandardMagenticManager(
            name=manager_name,
            description="データベース操作の全体調整を行うマネージャー",
            service=azure_completion_service,
            termination_strategy=None,
            max_iterations=15  # 繰り返し回数を制限
        )
        
        print(f"🔧 マネージャー '{manager.name}' を作成しました")
        
        # オーケストレーション作成
        orchestration = MagenticOrchestration(
            manager=manager,
            participants=agents
        )
        
        print("🚀 magenticオーケストレーションを開始します...")
        print("-" * 60)
        
        # 会話実行
        from semantic_kernel.contents.chat_history import ChatHistory
        history = ChatHistory()
        history.add_user_message(user_question)
        
        # コールバック関数でレスポンスを観察
        async def agent_response_callback(context) -> None:
            if hasattr(context, 'last_message') and context.last_message:
                speaker_name = getattr(context.last_message, 'name', 'Unknown')
                content = context.last_message.content or "（内容なし）"
                print(f"🎯 [{speaker_name}]:")
                print(f"   {content}")
                print("-" * 40)
        
        # オーケストレーション実行
        try:
            response = await orchestration.execute_async(
                conversation=history,
                agent_response_callback=agent_response_callback
            )
            
            print("🎉 オーケストレーション完了!")
            print("=" * 60)
            print("📝 最終結果:")
            
            # 最終結果の表示
            if response and len(response) > 0:
                final_message = response[-1]
                print(f"   {final_message.content}")
            else:
                print("   応答がありませんでした。")
                
            return response
            
        except Exception as inner_e:
            print(f"❌ オーケストレーション実行中の内部エラー: {str(inner_e)}")
            print(f"   エラータイプ: {type(inner_e).__name__}")
            
            # より詳細なエラー情報
            import traceback
            traceback.print_exc()
            return None
        
    except Exception as e:
        print(f"❌ セットアップエラー: {str(e)}")
        print(f"   エラータイプ: {type(e).__name__}")
        import traceback
        traceback.print_exc()
        return None

print("改良されたmagenticデータベースオーケストレーション関数が定義されました。")

エージェントレスポンスコールバック関数が定義されました。


# magenticオーケストレーション実行

In [14]:
async def run_magentic_database_orchestration(task: str):
    """
    magenticオーケストレーションを使用してマルチデータベースクエリを実行する関数
    
    Args:
        task (str): ユーザーのタスク（自然言語での質問）
    """
    
    # 1. magenticオーケストレーションを4体のエージェントとmagenticマネージャーで作成
    # StandardMagenticManagerは構造化出力をサポートするチャット補完モデルが必要
    magentic_orchestration = MagenticOrchestration(
        members=await create_database_agents(),
        manager=StandardMagenticManager(chat_completion_service=azure_completion_service),
        agent_response_callback=agent_response_callback,
    )

    # 2. ランタイムを作成して開始
    runtime = InProcessRuntime()
    runtime.start()

    # 3. タスクとランタイムでオーケストレーションを呼び出し
    print(f"🚀 Starting Magentic Orchestration for task: {task}")
    print("=" * 100)
    
    orchestration_result = await magentic_orchestration.invoke(
        task=task,
        runtime=runtime,
    )

    # 4. 結果を待機
    final_result = await orchestration_result.get()

    print("=" * 100)
    print(f"🎯 Final Result:")
    print(f"{final_result}")

    # 5. アイドル時にランタイムを停止
    await runtime.stop_when_idle()
    
    return final_result

print("magenticオーケストレーション実行関数が定義されました。")

magenticオーケストレーション実行関数が定義されました。


# 実行例とテスト

## PostgreSQL クエリの実行例

In [15]:
# PostgreSQLデータベースに対するクエリの実行例
postgres_task = "PostgreSQLデータベースに存在するすべてのテーブルの一覧と、各テーブルの基本的な構造を教えてください。"

result = await run_magentic_database_orchestration(postgres_task)

🚀 Starting Magentic Orchestration for task: PostgreSQLデータベースに存在するすべてのテーブルの一覧と、各テーブルの基本的な構造を教えてください。
**PostgresSQLGenerationAgent**

--------------------------------------------------------------------------------
🔧 Function Call: PostgresSchemaPlugin-get_tables
   Arguments: {}
**PostgresSQLGenerationAgent**

--------------------------------------------------------------------------------
📊 Function Result: PostgresSchemaPlugin-get_tables
   Result: {"tables": ["inventory", "order_details", "products", "users", "orders", "categories"]}
**PostgresSQLGenerationAgent**

--------------------------------------------------------------------------------
🔧 Function Call: PostgresSchemaPlugin-get_tables
   Arguments: {}
**PostgresSQLGenerationAgent**

--------------------------------------------------------------------------------
📊 Function Result: PostgresSchemaPlugin-get_tables
   Result: {"tables": ["inventory", "order_details", "products", "users", "orders", "categories"]}
**PostgresSQLGen

## CosmosDB クエリの実行例

In [16]:
# CosmosDB NoSQLデータベースに対するクエリの実行例
cosmos_task = "CosmosDBコンテナーの基本情報とサンプルドキュメントを取得して、データ構造を分析してください。"

result = await run_magentic_database_orchestration(cosmos_task)

🚀 Starting Magentic Orchestration for task: CosmosDBコンテナーの基本情報とサンプルドキュメントを取得して、データ構造を分析してください。
Fetching 5 sample documents from CosmosDB
Fetching 5 sample documents from CosmosDB
CosmosDB sample documents fetched successfully. Count: 5
**CosmosDBSQLGenerationAgent**

--------------------------------------------------------------------------------
🔧 Function Call: CosmosSchemaPlugin-get_container_info
   Arguments: {}
🔧 Function Call: CosmosSchemaPlugin-get_sample_documents
   Arguments: {"limit": 5}
**CosmosDBSQLGenerationAgent**

--------------------------------------------------------------------------------
📊 Function Result: CosmosSchemaPlugin-get_container_info
   Result: {"container": "tweets", "database": "twitterdb", "partition_key": {"paths": ["/tweet_id"], "kind": "Hash"}, "indexing_policy": {"indexingMode": "consistent", "automatic": true, "includedPaths": [{"path": "/*"}], "excludedPaths": [{"path": "/\"_etag\"/?"}], "fullTextIndexes": []}}
**CosmosDBSQLGenerationAgent**

-

## 複合データベースクエリの実行例

In [18]:
# 複合タスク（PostgreSQLとCosmosDB両方を使用）のテスト
complex_task = """
次の分析を行ってください：
1. PostgreSQLから都道府県データを取得して概要を教えてください
2. CosmosDBからサンプルドキュメントを取得して構造を教えてください
3. 両方のデータベースの情報をまとめて比較してください
"""

print("🔍 複合タスクを実行します...")
print(f"📋 タスク内容:\n{complex_task}")
print("=" * 80)

try:
    # より安全なアプローチ：段階的に実行
    print("🛠️  段階的実行を開始します...")
    
    # まず簡単なタスクで確認
    simple_test = "PostgreSQLのテーブル一覧を教えてください"
    result = await run_magentic_database_orchestration(simple_test)
    
    if result:
        print("✅ 簡単なタスクが成功しました。複合タスクに進みます...")
        print("-" * 80)
        
        # 複合タスク実行
        complex_result = await run_magentic_database_orchestration(complex_task)
        
        if complex_result:
            print("🎉 複合タスクが正常に完了しました!")
        else:
            print("⚠️  複合タスクは失敗しましたが、エラーは処理されました。")
    else:
        print("❌ 簡単なタスクが失敗したため、複合タスクをスキップします。")
        
except Exception as e:
    print(f"❌ 実行中にエラーが発生しました: {str(e)}")
    print(f"   エラータイプ: {type(e).__name__}")
    
    # 詳細なエラー情報
    import traceback
    traceback.print_exc()
    
print("\n" + "=" * 80)
print("🏁 複合タスクテストが完了しました。")

🔍 複合タスクを実行します...
📋 タスク内容:

次の分析を行ってください：
1. PostgreSQLから都道府県データを取得して概要を教えてください
2. CosmosDBからサンプルドキュメントを取得して構造を教えてください
3. 両方のデータベースの情報をまとめて比較してください

🛠️  段階的実行を開始します...
🚀 Starting Magentic Orchestration for task: PostgreSQLのテーブル一覧を教えてください
🚀 Starting Magentic Orchestration for task: PostgreSQLのテーブル一覧を教えてください
**PostgresSQLGenerationAgent**

--------------------------------------------------------------------------------
🔧 Function Call: PostgresSchemaPlugin-get_tables
   Arguments: {}
**PostgresSQLGenerationAgent**

--------------------------------------------------------------------------------
📊 Function Result: PostgresSchemaPlugin-get_tables
   Result: {"tables": ["inventory", "order_details", "products", "users", "orders", "categories"]}
**PostgresSQLGenerationAgent**

--------------------------------------------------------------------------------
🔧 Function Call: PostgresSchemaPlugin-get_tables
   Arguments: {}
**PostgresSQLGenerationAgent**

----------------------------------------

# まとめ

## このノートブックで実装したもの

このノートブックでは、**magenticオーケストレーション**を使用して、PostgreSQLとCosmosDB NoSQLの両方に対応する**4体のエージェントシステム**を実装しました：

### 実装されたエージェント

1. **PostgresSQLGenerationAgent** - PostgreSQL用SQLクエリ生成エージェント
   - テーブル一覧とスキーマ情報を取得してSQLクエリを生成
   - PostgresSchemaPluginを使用してデータベース構造を理解

2. **PostgresSQLExecutionAgent** - PostgreSQL用SQLクエリ実行エージェント
   - 生成されたSQLクエリを実際にPostgreSQLで実行
   - PostgresQueryPluginを使用してクエリ実行と結果取得

3. **CosmosDBSQLGenerationAgent** - CosmosDB NoSQL用SQLクエリ生成エージェント
   - コンテナー情報とサンプルドキュメントを取得してSQLクエリを生成
   - CosmosSchemaPluginを使用してドキュメント構造を理解

4. **CosmosDBSQLExecutionAgent** - CosmosDB NoSQL用SQLクエリ実行エージェント
   - 生成されたSQLクエリを実際にCosmosDBで実行
   - CosmosQueryPluginを使用してクエリ実行と結果取得

### 主な特徴

- **magenticオーケストレーション**: Microsoftの最新のマルチエージェント協調システムを使用
- **自動的なエージェント選択**: StandardMagenticManagerが適切なエージェントを自動選択
- **マルチデータベース対応**: PostgreSQL（リレーショナル）とCosmosDB（NoSQL）の両方に対応
- **スキーマ認識**: 各データベースの構造を動的に取得して適切なクエリを生成
- **エラーハンドリング**: 各エージェントが独立してエラーを処理

### 使用方法

```python
# 任意のデータベースクエリタスクを自然言語で指定
task = "ユーザーテーブルから最新の10件のデータを取得してください"
result = await run_magentic_database_orchestration(task)
```

magenticオーケストレーションにより、ユーザーは**どのデータベースを使用するかを明示的に指定する必要がなく**、エージェント間の自動協調によって適切なデータベースとクエリが選択されます。